In [51]:
import requests
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import timedelta
# Google Cloud dependencies
import google.auth
from google.cloud import bigquery
#import datalab.storage as gcs

beginning_date = "\"2020-06-06\""
end_date = "\"2020-07-06\""




In [52]:
query = "WITH ticket_count as (SELECT \
REGEXP_REPLACE(classless_journey_map,   r\"[0-9_]\", \"\") as classless_journey_map ,COUNT(ticket_id) as count, nb_legs \
FROM `matching.X6` \
WHERE dt_of_issue > " + beginning_date + " AND dt_of_issue < " +  end_date + " AND price IS NULL AND nb_legs IN (1, 2, 4) AND leadtime <35  \
GROUP BY classless_journey_map, nb_legs) SELECT * FROM ticket_count WHERE count >10"

In [53]:
print(query)

WITH ticket_count as (SELECT REGEXP_REPLACE(classless_journey_map,   r"[0-9_]", "") as classless_journey_map ,COUNT(ticket_id) as count, nb_legs FROM `matching.X6` WHERE dt_of_issue > "2020-06-06" AND dt_of_issue < "2020-07-06" AND price IS NULL AND nb_legs IN (1, 2, 4) AND leadtime <35  GROUP BY classless_journey_map, nb_legs) SELECT * FROM ticket_count WHERE count >10


In [54]:
query = "SELECT * FROM `skytra-benchmark-rnd.scratch_PaR.kiwi_api_lookout` "


#loads a list of itineraries to search
client = bigquery.Client()

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="EU",)  # API request - starts the query

ListUnmatchedItineraries_DF = query_job.result().to_dataframe()



ListUnmatchedItineraries_DF = ListUnmatchedItineraries_DF.sort_values(by='count', ascending=False,)
#we are limited to 100,000 requests per day, and we are searching each itinerary 4 times
ListUnmatchedItineraries_DF = ListUnmatchedItineraries_DF[0:25000]

ListUnmatchedItineraries_DF["return"] = np.where(ListUnmatchedItineraries_DF["nb_legs"]  ==1,
                                                    "oneway",
                                                    "round")

#setting the to and from locations
ListUnmatchedItineraries_DF["From"] = ListUnmatchedItineraries_DF["classless_journey_map"].str[0:3]
ListUnmatchedItineraries_DF["To"] = np.where(ListUnmatchedItineraries_DF["nb_legs"]  ==1,
                                             ListUnmatchedItineraries_DF["classless_journey_map"].str[4:7],
                                             "")

ListUnmatchedItineraries_DF["To"] = np.where(ListUnmatchedItineraries_DF["nb_legs"]  ==2,
                                             ListUnmatchedItineraries_DF["classless_journey_map"].str[4:7],
                                             ListUnmatchedItineraries_DF["To"])

ListUnmatchedItineraries_DF["To"] = np.where(ListUnmatchedItineraries_DF["nb_legs"]  ==4,
                                             ListUnmatchedItineraries_DF["classless_journey_map"].str[12:15],
                                             ListUnmatchedItineraries_DF["To"])

list_of_itineraries = list()
for i in ListUnmatchedItineraries_DF.index:
    list_of_itineraries.append(ListUnmatchedItineraries_DF.From[i] + "&to=" +  ListUnmatchedItineraries_DF.To[i])


list_of_flight_type = list(ListUnmatchedItineraries_DF["return"]) 

ListUnmatchedItineraries_DF.head(30)

,classless_journey_map,count,nb_legs,return,From,To
3225,LXA-CTU,29738,1,oneway,LXA,CTU
7094,URC-XIY,26010,1,oneway,URC,XIY
2941,KHG-URC,23845,1,oneway,KHG,URC
3001,URC-KHG,22822,1,oneway,URC,KHG
8069,CGO-SZX,19651,1,oneway,CGO,SZX
8140,URC-CGO,19398,1,oneway,URC,CGO
1584,CAN-CKG,16660,1,oneway,CAN,CKG
2798,CKG-CAN,15651,1,oneway,CKG,CAN
9187,PEK-SHA,15103,1,oneway,PEK,SHA
7885,SHA-PEK,15035,1,oneway,SHA,PEK


In [55]:
ListUnmatchedItineraries_DF.tail(30)

,classless_journey_map,count,nb_legs,return,From,To
9265,"SIP-SVO,SVO-MCX",11,2,round,SIP,SVO
9278,"BCN-FRA,FRA-ADD",11,2,round,BCN,FRA
9277,"LYS-BRU,BRU-VNO",11,2,round,LYS,BRU
9276,"LIS-FRA,FRA-MXP",11,2,round,LIS,FRA
9275,"BCN-FRA,FRA-HAJ",11,2,round,BCN,FRA
9274,"TFS-MAD,MAD-IBZ",11,2,round,TFS,MAD
9273,"ZHA-KMG,KMG-SYM",11,2,round,ZHA,KMG
9272,"BEG-MUC,MUC-AMS",11,2,round,BEG,MUC
9271,"BRU-MAD,MAD-IBZ",11,2,round,BRU,MAD
9270,"KCA-URC,URC-CAN",11,2,round,KCA,URC


In [56]:
list_of_flight_type = list(ListUnmatchedItineraries_DF["return"]) 

#for each itinerary we will do 4 weekly searches
date1 = (date.today() + timedelta(days=2)).strftime("%d/%m/%Y")
date2 = (datetime.datetime.strptime(date1, "%d/%m/%Y") + timedelta(days=7)).strftime("%d/%m/%Y")
date3 = (datetime.datetime.strptime(date2, "%d/%m/%Y")+ timedelta(days=7)).strftime("%d/%m/%Y")
date4 = (datetime.datetime.strptime(date3, "%d/%m/%Y")+ timedelta(days=7)).strftime("%d/%m/%Y")
date5 = (datetime.datetime.strptime(date4, "%d/%m/%Y")+ timedelta(days=7)).strftime("%d/%m/%Y")

In [57]:
date5

'12/08/2020'

In [ ]:

#sets a list of usernames to chose later on
username_tuple = ("ab747")

for itinerary in list_of_itineraries:

    #sets our search id, so we can identify our searches later on
    partnerID = "ab747"
    #defining the type of flight
    fligth_type = list_of_flight_type[list_of_itineraries.index(itinerary)]
        
    url1 = "https://api.skypicker.com/flights?flyFrom=" + itinerary + "&dateFrom=" + date1 + "&dateTo=" + date2 + "&partner=" + partnerID #+ "&flight_type=" + fligth_type + "&direct_flights=0"+"&max_stopovers=2"+"&limit=2000" 
    url2 = "https://api.skypicker.com/flights?flyFrom=" + itinerary + "&dateFrom=" + date2 + "&dateTo=" + date3 + "&partner=" + partnerID #+ "&flight_type=" + fligth_type + "&direct_flights=0"+"&max_stopovers=2"+"&limit=2000" 
    url3 = "https://api.skypicker.com/flights?flyFrom=" + itinerary + "&dateFrom=" + date3 + "&dateTo=" + date4 + "&partner=" + partnerID #+ "&flight_type=" + fligth_type + "&direct_flights=0"+"&max_stopovers=2"+"&limit=2000" 
    url4 = "https://api.skypicker.com/flights?flyFrom=" + itinerary + "&dateFrom=" + date4 + "&dateTo=" + date5 + "&partner=" + partnerID #+ "&flight_type=" + fligth_type + "&direct_flights=0"+"&max_stopovers=2"+"&limit=2000" 
        
    #we will get the data from the requests on the KIWI dataset, so no need to save the results
    print("1")
    try:
        r = requests.get(url1)
        print(url1)
    except:
        #sometimes the itinerary does not exist in Kiwi
        pass
        #repeat for the other 3 weekly searches

    print("2")
    try:
        r = requests.get(url2)
        print(url2)
    except:
        pass

    print("3")
    try:
        r = requests.get(url3)
        print(url3)
    except:
        pass

    print("4")
    try:
        r = requests.get(url4)
        print(url4)
    except:
        pass

1
https://api.skypicker.com/flights?flyFrom=LXA&to=CTU&dateFrom=15/07/2020&dateTo=22/07/2020&partner=ab747
2
https://api.skypicker.com/flights?flyFrom=LXA&to=CTU&dateFrom=22/07/2020&dateTo=29/07/2020&partner=ab747
3
https://api.skypicker.com/flights?flyFrom=LXA&to=CTU&dateFrom=29/07/2020&dateTo=05/08/2020&partner=ab747
4
https://api.skypicker.com/flights?flyFrom=LXA&to=CTU&dateFrom=05/08/2020&dateTo=12/08/2020&partner=ab747
1
https://api.skypicker.com/flights?flyFrom=URC&to=XIY&dateFrom=15/07/2020&dateTo=22/07/2020&partner=ab747
2
https://api.skypicker.com/flights?flyFrom=URC&to=XIY&dateFrom=22/07/2020&dateTo=29/07/2020&partner=ab747
3
https://api.skypicker.com/flights?flyFrom=URC&to=XIY&dateFrom=29/07/2020&dateTo=05/08/2020&partner=ab747
4
https://api.skypicker.com/flights?flyFrom=URC&to=XIY&dateFrom=05/08/2020&dateTo=12/08/2020&partner=ab747
1
https://api.skypicker.com/flights?flyFrom=KHG&to=URC&dateFrom=15/07/2020&dateTo=22/07/2020&partner=ab747
2
https://api.skypicker.com/flights?f